In [ ]:
import requests, json, time
from pathlib import Path
import osm2geojson
from shapely.geometry import shape
import geopandas as gpd


In [ ]:
# Sofia coords
MIN_LAT, MAX_LAT = 42.55, 42.85
MIN_LON, MAX_LON = 23.10, 23.45

# Tiles
N_ROWS = 3
N_COLS = 3

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

out_dir = Path("osm_sofia_sidewalks")
out_dir.mkdir(exist_ok=True)
merged_geojson = out_dir / "sofia_sidewalks.geojson"

In [ ]:
# Tiles

lats = [MIN_LAT + i*(MAX_LAT-MIN_LAT)/N_ROWS for i in range(N_ROWS+1)]
lons = [MIN_LON + j*(MAX_LON-MIN_LON)/N_COLS for j in range(N_COLS+1)]

tiles = []
for i in range(N_ROWS):
    for j in range(N_COLS):
        tiles.append((lats[i], lons[j], lats[i+1], lons[j+1]))


In [ ]:
def overpass_query(bbox):
    """bbox = (minlat, minlon, maxlat, maxlon)"""
    minlat, minlon, maxlat, maxlon = bbox

    query = f"""
    [out:json][timeout:120];
    (
      way["highway"="footway"]["footway"="sidewalk"]({minlat},{minlon},{maxlat},{maxlon});
      way["sidewalk"]({minlat},{minlon},{maxlat},{maxlon});
      way["highway"="path"]["foot"="designated"]({minlat},{minlon},{maxlat},{maxlon});
    );
    out body;
    >;
    out skel qt;
    """

    r = requests.post(OVERPASS_URL, data={"data": query})
    if not r.ok:
        print("ERROR:", r.status_code, r.text[:200])
        return None
    return r.json()

In [ ]:
geo_features = []

for idx, bbox in enumerate(tiles):
    print(f"Tile {idx+1}/{len(tiles)} → {bbox}")

    data = overpass_query(bbox)
    if data is None:
        print("Failed tile, skipping")
        continue

    # convert to geojson
    gj = osm2geojson.json2geojson(data)

    # extract features
    features = gj.get("features", [])
    print(f" → {len(features)} features")

    geo_features.extend(features)

    time.sleep(1)

In [ ]:
# get all features to see what is avalible
max_feature = max(geo_features, key=lambda f: len(f["properties"].get("tags", {})))


In [ ]:
max_feature

In [ ]:
print(f"Total collected features: {len(geo_features)}")

flat_features = []
for feat in geo_features:
    props = feat.get("properties", {}).copy()

    # flatten tags
    tags = props.pop("tags", {}) or {}
    for k, v in tags.items():
        # if key already exists in properties, prefix with 'tag_'
        if k in props:
            props[f"tag_{k}"] = v
        else:
            props[k] = v

    # write new feature
    flat_features.append({
        "type": "Feature",
        "properties": props,
        "geometry": feat.get("geometry")
    })

# make final GeoJSON
merged = {
    "type": "FeatureCollection",
    "features": flat_features
}

with merged_geojson.open("w", encoding="utf8") as f:
    json.dump(merged, f, ensure_ascii=False, indent=2)

print(f"SAVED → {merged_geojson}")


# test load with geopandas
try:
    gdf = gpd.GeoDataFrame.from_features(merged["features"])
    print("GDF load OK!")
    print(gdf.head())
except Exception as e:
    print("Failed geopandas load:", e)
